# Chapter 20. Concurrent Execution

This chapter focuses on `concurrent.futures.Executor` classes that encapuslate the pattern of "spawning a bunch of independent threads and collecting the results in a queue"

Here the author introduces the concept of "futures"--objects representing the asynchronous execution of an operation, similar to JS promises.

Executors are the most important high-level feature while futures are low-level objects.

## Concurrent Web Downloads

Concurrency is essential for efficient network I/O: instead of idly waiting for remote machines, the application should do sth else until a response comes back.

Three simples programs to download images of 20 country flags from the web
 - `flags.py`: runs sequentially
 - `flags_threadpool.py` uses the `concurrent.futures` pacakge
 - `flags_asyncio.py` uses the `asyncio`

### A Sequential Download Script

In [ ]:
!pip install httpx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00


In [ ]:
import time
from pathlib import Path
from typing import Callable

import httpx

POP20_CC = ('CN IN US ID BR PK NG BD RU KR '
            'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'https://www.fluentpython.com/data/flags'
DEST_DIR = Path('downloaded')

# save the img bytes to filename in the DEST_DIR
def save_flag(img: bytes, filenames: str) -> None:
  (DEST_DIR / filename).write_bytes(img)

def get_flag(cc: str) -> bytes:
  url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
  resp = httpx.get(url, timeout=6.1, # it's a good practice to add a sensible timeout
                   follow_redirects=True) # By default, HTTPX does not follow redirects
  resp.raise_for_status()
  return resp.content

# key function to compare with the concurrent implementations
def download_many(cc_list: list[str]) -> int:
  for cc in sorted(cc_list):
    image = get_flag(cc)
    save_flag(image, f'{cc}.gif')
    print(cc, end=" ", flush=True) # display one country code at a time in the same line
                                   # flush=True is needed because by default Python output is line buffered
  return len(cc_list)

def main(downloader: Callable[[list[str]], int]) -> None:
  DEST_DIR.mkdir(exist_ok=True)
  t0 = time.perf_counter()
  count = downloader(POP20_CC)
  elapsed = time.perf_counter() - t0
  print(f"\n{count} downloads in {elapsed:.2f}s")

if __name__ == "__main__":
  main(download_many)


In [ ]:
!python flags.py

BD BR CD CN DE EG ET FR ID IN IR KR MX NG PH PK RU TR US VN 
20 downloads in 6.43s


### Downloading with concurrent.futures

Main Features: `ThreadPoolExecutor` and `ProcessPoolExectuor` classes, which implement an API to submit cllables for execution in different threads or processes. The classes transparently manage a pool of worker threads or processes, and queues to distribute jobs and collect results.

In [ ]:
# flags_threadpool.py
from concurrent import futures

from flags import save_flag, get_flag, main

def download_one(cc: str):
  image = get_flag(cc)
  save_flag(image, f'{cc}.gif')
  print(cc, end=" ", flush=True)
  return cc

def download_many(cc_list: list[str]) -> int:
  # instantiate the thread pool executor as a context manager
  # exectuor.__exit__ will call executor.shutdown(wait=True) which will block until all threads are done
  with futures.ThreadPoolExecutor() as executor:
    res = executor.map(download_one, sorted(cc_list))

  return len(list(res))

if __name__ == "__main__":
  main(download_many)



In [ ]:
!python flags_threadpool.py

BD CN DE BR EG CD ET ID FR KR IN IR MX NG PH RU PK TR US VN 
20 downloads in 1.29s


In [ ]:
import os
os.cpu_count() + 4

6

### Where are teh Futures?

Futures are core components of `concurrent.futures` and of `asyncio` but as users of these libraries we sometimes don't see them.

There are two classes naemd `Future` in the standard library: `concurrent.futures.Future` and `asyncio.Future`. They serve the same purpose: an instance of either `Future` class represents a deferred computation that may or may not have completed.

Futures encapsulate pending operations so that we can put them in queues, check whether they are done, and retrieve results when they become available.

We should not create them: they are meant to be instantiated exclusively by the concurrency framework, be it `concurrent.futures` or `asyncio`. Why? a `Future` represents something that will eventually run, therefore it must be scheduled to run, and that's the job of the framework.

Application code is not supposed to change the state of the a future: the concurrency framework changes the state of a future when the computation it represents is done, and we can't control when that happens.

Both types of `Future` have a `.done()` method that is nonblocking and returns a Boolean that tells you whether the callable wrapped by that future has executed or not. However, instaed of repeatedly asking whether a future is done, client code usually asks to be notified. That's why both `Future` classes have an `.add_done_callback()` method.

There is also a `.result()` method, which works the same in both classes when the future is done: it returns the result of the callable, or re-raises whatever exception might have been thrown when the callable was executed. However, when the future is not done, the behavior of the `result` method is very different.

In [ ]:
# flags_threadpool_futures.py
from concurrent import futures

from flags import save_flag, get_flag, main

def download_one(cc: str):
  image = get_flag(cc)
  save_flag(image, f'{cc}.gif')
  print(cc, end=" ", flush=True)
  return cc

def download_many(cc_list: list[str]) -> int:

  cc_list = cc_list[:5] # just for this demonstration
  # instantiate the thread pool executor as a context manager
  # exectuor.__exit__ will call executor.shutdown(wait=True) which will block until all threads are done
  with futures.ThreadPoolExecutor(max_workers=3) as executor: # max_workers=3 to see pending futures in the output
    to_do: list[futures.Future] = []
    for cc in sorted(cc_list):
      future = executor.submit(download_one, cc)
      to_do.append(future)
      print(f"Scheduled for {cc}: {future}")

    for count, future in enumerate(futures.as_completed(to_do), 1):
      res: str = future.result()
      print(f"{future} result: {res!r}")

  return count

if __name__ == "__main__":
  main(download_many)



`concurrent.futures.as_completed` function takes an iterable of futures and returns an iterator that yields futures as they are done.

The higher-level `executor.map` is replaced by two `for` loops: one to create and schedule the futures, the other to retrieve their results. While we are at it, we'll add a few `print` calls to display each future before and after it's done.

In [ ]:
!python flags_threadpool_futures.py

Scheduled for BR: <Future at 0x799001ff6260 state=running>
Scheduled for CN: <Future at 0x799001d70c10 state=running>
Scheduled for ID: <Future at 0x799001d71480 state=running>
Scheduled for IN: <Future at 0x799001d71d80 state=pending>
Scheduled for US: <Future at 0x799001d71db0 state=pending>
CN <Future at 0x799001d70c10 state=finished returned str> result: 'CN'
ID <Future at 0x799001d71480 state=finished returned str> result: 'ID'
BR <Future at 0x799001ff6260 state=finished returned str> result: 'BR'
IN <Future at 0x799001d71d80 state=finished returned str> result: 'IN'
US <Future at 0x799001d71db0 state=finished returned str> result: 'US'

5 downloads in 0.35s


Now, let's take a brief look at a simple way to work around the GIL for CPU-bound jobs using `concurrent.futures`.

## Launching Processes with concurrent.futures

The real value of `ProcessPoolExecutor` is in CPU-intensive jobs.

In [ ]:
# proc_pool.py

import sys
from concurrent import futures # hides multiprocessing / SimpleQueue / etc
from time import perf_counter
from typing import NamedTuple

from primes import is_prime, NUMBERS

class PrimeResult(NamedTuple):
  n: int
  flag: bool
  elapsed: float

def check(n: int) -> PrimeResult:
  t0 = perf_counter()
  res = is_prime(n)
  return PrimeResult(n, res, perf_counter() - t0)

def main() -> None:
  if len(sys.argv) < 2:
    workers = None
  else:
    workers = int(sys.argv[1])

  executor = futures.ProcessPoolExecutor(workers)
  actual_workers = executor._max_workers # type: ignore

  print(f"Checking {len(NUMBERS)} numbers with {actual_workers} processes:")

  t0 = perf_counter()

  numbers = sorted(NUMBERS, reverse=True)
  with executor:
    # executor.map() returns the result in the same order as the numbers are given
    for n, prime, elapsed in executor.map(check, numbers):
      label = 'P' if prime else ' '
      print(f"{n: 16}   {label} {elapsed:9.6f}s")

  time = perf_counter() - t0
  print(f"Total time: {time:.2f}s")

if __name__ == "__main__":
  main()

In [ ]:
!python proc_pool.py

Checking 20 numbers with 2 processes:
 9999999999999999      0.000036s
 9999999999999917   P 15.200927s
 7777777777777777      0.000005s
 7777777777777753   P 13.473852s
 7777777536340681     13.214291s
 6666667141414921     12.325249s
 6666666666666719   P 12.355056s
 6666666666666666      0.000002s
 5555555555555555      0.000009s
 5555555555555503   P 11.451030s
 5555553133149889     12.853707s
 4444444488888889     11.632231s
 4444444444444444      0.000002s
 4444444444444423   P  9.667734s
 3333335652092209      8.738373s
 3333333333333333      0.000010s
 3333333333333301   P  6.512187s
 299593572317531   P  2.902751s
 142702110479723   P  1.590411s
               2   P  0.000002s
Total time: 66.87s


## Experimenting with `Executor.map`


Experiment that may help us visualize the operation of `Executor.map`

In [ ]:
# demo_executor_map.py

from time import sleep, strftime
from concurrent import futures

def display(*args):
  """this function simply prints whatever argument it gets, preceded by a timestamp in [HH:MM:SS]"""
  print(strftime('[%H:%M:%S]'), end=' ')
  print(*args)

def loiter(n):
  """loiter does nothing except display a message when it starts, sleep for n seconds, then display a message when it ends"""
  msg = '{}loiter({}): doing nothing for {}s'
  display(msg.format('\t' * n, n, n))
  sleep(n)
  msg = '{}loiter({}): done.'
  display(msg.format('\t'*n, n))
  return n * 10

def main():
  display('Script starting')
  executor = futures.ThreadPoolExecutor(max_workers=3)
  results = executor.map(loiter, range(5))
  display('results:', results)
  display('waiting for individual results:')

  # enumerate call in the for loop will implicitly invoke next(results)
  # which in turn will invoke _f.result() on the _f future representing the first call
  for i, result in enumerate(results):
    display(f"result {i}: {result}")

if __name__ == "__main__":
  main()



[02:46:39] Script starting
[02:46:39][02:46:39] 	loiter(1): doing nothing for 1s
[02:46:39][02:46:39] results: <generator object Executor.map.<locals>.result_iterator at 0x7f1951a62ab0>
[02:46:39] waiting for individual results:
 		loiter(2): doing nothing for 2s
 loiter(0): doing nothing for 0s
[02:46:39] loiter(0): done.
[02:46:39] 			loiter(3): doing nothing for 3s
[02:46:39] result 0: 0
[02:46:40] 	loiter(1): done.
[02:46:40] 				loiter(4): doing nothing for 4s
[02:46:40] result 1: 10
[02:46:41] 		loiter(2): done.
[02:46:41] result 2: 20
[02:46:42] 			loiter(3): done.
[02:46:42] result 3: 30
[02:46:44] 				loiter(4): done.
[02:46:44] result 4: 40


In [ ]:
!python demo_executor_map.py

[02:46:44] Script starting
[02:46:44] loiter(0): doing nothing for 0s
[02:46:44] loiter(0): done.
[02:46:44] 	loiter(1): doing nothing for 1s
[02:46:44] 		loiter(2): doing nothing for 2s
[02:46:44] 			loiter(3): doing nothing for 3s
[02:46:44] results: <generator object Executor.map.<locals>.result_iterator at 0x7f5173e73d10>
[02:46:44] waiting for individual results:
[02:46:44] result 0: 0
[02:46:45] 	loiter(1): done.
[02:46:45] 				loiter(4): doing nothing for 4s
[02:46:45] result 1: 10
[02:46:46] 		loiter(2): done.
[02:46:46] result 2: 20
[02:46:47] 			loiter(3): done.
[02:46:47] result 3: 30
[02:46:49] 				loiter(4): done.
[02:46:49] result 4: 40


The `Executor.map` function is easy to use, but often it's perferable to get the results as they are ready, regardless of the order they were submitted. To do that, we need a combination of the `Executor.submit` method and the `future.as_completed` function.

## Downloads with Progress Display and Error Handling

`flags2_common.py` contains common functions and settings used by all flags2 examples, including a `main` function, which takes care of command-line parsing, timing, and reporting results.

In [ ]:
# flags2_common.py

import argparse
import string
import sys
import time
from collections import Counter
from enum import Enum
from pathlib import Path

DownloadStatus = Enum('DownloadStatus', 'OK NOT_FOUND ERROR')

POP20_CC = ('CN IN US ID BR PK NG BD RU KR '
            'MX PH VN ET EG DE IR TR CD FR').split()

DEFAULT_CONCUR_REQ = 1
MAX_CONCUR_REQ = 1

SERVERS = {
    'REMOTE': 'https://www.fluentpython.com/data/flags',
    'LOCAL':  'http://localhost:8000/flags',
    'DELAY':  'http://localhost:8001/flags',
    'ERROR':  'http://localhost:8002/flags',
}
DEFAULT_SERVER = 'LOCAL'

DEST_DIR = Path('downloaded')
COUNTRY_CODES_FILE = Path('country_codes.txt')

def save_flag(img: bytes, filename: str) -> None:
  (DEST_DIR / filename).write_bytes(img)

def initial_report(cc_list: list[str],
                   actual_req: int,
                   server_label: str) -> None:
  if len(cc_list) <= 10:
    cc_msg = ', '.join(cc_list)
  else:
    cc_msg = f'from {cc_list[0]} to {cc_list[-1]}'
  print(f'{server_label} site: {SERVERS[server_label]}')
  plural = 's' if len(cc_list) != 1 else ''
  print(f'Searching for {len(cc_list)} flag{plural}: {cc_msg}')

  if actual_req == 1:
    print('1 connection will be used')
  else:
    print(f'{actual_req} concurrent connections will be used.')

def final_report(cc_list: list[str],
                 counter: Counter[DownloadStatus],
                 start_time: float) -> None:
  elapsed = time.perf_counter() - start_time
  print('-' * 20)
  plural = 's' if counter[DownloadStatus.OK] != 1 else ''
  print(f'{counter[DownloadStatus.OK]:3} flag{plural} downloaded.')
  if counter[DownloadStatus.NOT_FOUND]:
    print(f'{counter[DownloadStatus.NOT_FOUND]:3} not found.')
  if counter[DownloadStatus.ERROR]:
    plural = 's' if counter[DownloadStatus.ERROR] != 1 else ''
    print(f'{counter[DownloadStatus.ERROR]:3} error{plural}.')
  print(f'Elapsed time: {elapsed:.2f}s')

def expand_cc_args(every_cc: bool,
                   all_cc: bool,
                   cc_args: list[str],
                   limit: int) -> list[str]:
  codes: set[str] = set()
  A_Z = string.ascii_uppercase
  if every_cc:
    codes.update(a + b for a in A_Z for b in A_Z)
  elif all_cc:
    text = COUNTRY_CODES_FILE.read_text()
    codes.update(text.split())
  else:
    for cc in (c.upper() for c in cc_args):
      if len(cc) == 1 and cc in A_Z:
        codes.update(cc + c for c in A_Z)
      elif len(cc) == 2 and all(c in A_Z for c in cc):
        codes.add(cc)
      else:
        raise ValueError('*** Usage error: each CC argument '
                          'must be A to Z or AA to ZZ.')
  return sorted(codes)[:limit]

def process_args(default_concur_req):
  server_options = ', '.join(sorted(SERVERS))
  parser = argparse.ArgumentParser(
      description='Download flags for country codes. '
                  'Default: top 20 countries by population'
  )

  parser.add_argument(
      'cc', metavar='CC', nargs='*',
      help='country code or 1st letter (eg. B for BA...BZ)'
  )

  parser.add_argument(
      '-a', '--all', action='store_true',
      help='get all available flags (AD to ZW)'
  )

  parser.add_argument(
      '-e', '--every', action='store_true',
      help='get flags for every possible code (AA ... AZ)'
  )

  parser.add_argument(
      '-l', '--limit', metavar='N', type=int, help='limit to N first codes',
      default=sys.maxsize
  )

  parser.add_argument(
      '-m', '--max_req', metavar='CONCURRENT' , type=int,
      default=default_concur_req,
      help=f'maximum concurrent requests (default={default_concur_req})'
  )

  parser.add_argument(
      '-s', '--server', metavar='LABEL', default=DEFAULT_SERVER,
      help=f"Server to hit; one of {server_options} "
           f"(default={DEFAULT_SERVER})"
  )

  parser.add_argument(
      '-v', '--verbose', action='store_true',
      help='output detailed progress info'
  )

  args = parser.parse_args()

  if args.max_req < 1:
    print('*** Usage error: --mx_req CONCURRENT must be >= 1')
    parser.print_usage()
    sys.exit(2)
  if args.limit < 1:
    print('*** Usage error: --limit N must be >= 1')
    parser.print_usage()
    sys.exit(2)

  args.server = args.server.upper()
  if args.server not in SERVERS:
    print(f"*** Usage error: --server LABEL "
          f"must be one of {server_options}")
    parser.print_usage()
    sys.exit(2)

  try:
    cc_list = expand_cc_args(args.every, args.all, args.cc, args.limit)
  except ValueError as err:
    print(err.args[0])
    parser.print_usage()
    sys.exit(2)

  if not cc_list:
    cc_list = sorted(POP20_CC)[:args.limit]
  return args, cc_list

def main(download_many, default_concur_req, max_concur_req):
  args, cc_list = process_args(default_concur_req)
  actual_req = min(args.max_req, max_concur_req, len(cc_list))
  initial_report(cc_list, actual_req, args.server)
  base_url = SERVERS[args.server]
  DEST_DIR.mkdir(exist_ok=True)
  t0 = time.perf_counter()
  counter = download_many(cc_list, base_url, args.verbose, actual_req)
  final_report(cc_list, counter, t0)



In [ ]:
import time
from tqdm import tqdm
for i in tqdm(range(1000)):
  time.sleep(0.01)

100%|██████████| 1000/1000 [00:10<00:00, 97.32it/s]


In [ ]:
# from google.colab import files
!wget "https://github.com/fluentpython/example-code-2e/raw/master/20-executors/getflags/flags.zip"

In [ ]:
!unzip flags.zip -d .

In [ ]:
# flags2_sequential.py

from collections import Counter
from http import HTTPStatus

import httpx
import tqdm

from flags2_common import main, save_flag, DownloadStatus

DEFAULT_CONCUR_REQ = 1
MAX_CONCUR_REQ = 1

def get_flag(base_url: str, cc: str) -> bytes:
  url = f'{base_url}/{cc}/{cc}.gif'.lower()
  resp = httpx.get(url, timeout=3.1, follow_redirects=True)
  resp.raise_for_status() # Raises HTTPStatusError
                          # if the HTTP status code is not is range(200, 300)
  return resp.content

def download_one(cc: str, base_url: str, verbose: bool = False) -> DownloadStatus:
  try:
    image = get_flag(base_url, cc)
  except httpx.HTTPStatusError as exc:
    res = exc.response
    if res.status_code == HTTPStatus.NOT_FOUND:
      # catches HTTPStatusError to handle HTTP code 404
      # by setting its local status to DownloadStatus.NOT_FOUND
      status = DownloadStatus.NOT_FOUND
      msg = f"not found: {res.url}"
    else:
      raise
  else:
    save_flag(image, f'{cc}.gif')
    status = DownloadStatus.OK
    msg = 'OK'

  if verbose:
    print(cc, msg)

  return status

def download_many(cc_list: list[str], base_url: str,
                  verbose: bool, _unused_concur_req: int) -> Counter[DownloadStatus]:
  counter: Counter[DownloadStatus] = Counter()
  cc_iter = sorted(cc_list)
  if not verbose:
    cc_iter = tqdm.tqdm(cc_iter)
  for cc in cc_iter:
    try:
      status = download_one(cc, base_url, verbose)
    # Exceptions raised by get_flag and not handled by download_one are handled here
    except httpx.HTTPStatusError as exc:
      error_msg = 'HTTP error {resp.status_code} - {resp.reason_phrase}'
      error_msg = error_msg.format(resp=exc.response)
    # Other network-related exceptions are handled here
    except httpx.RequestError as exc:
      error_msg = f'{exc} {type(exc)}'.strip()
    # Exit the loop if the user hits Ctrl-C
    except KeyboardInterrupt:
      break
    # If no exception escaped download_one, clear the error msg
    else:
      error_msg = ''

    if error_msg:
      status = DownloadStatus.ERROR
    counter[status] += 1
    if verbose and error_msg:
      print(f'{cc} error: {error_msg}')

  return counter

### Using futures.as_completed

In order to integrate the `tqdm` progress bar and handle errors on each request, the `flags2_threadpool.py` script uses `futures.ThreadpoolExecutor` with the `futures.as_completed` function we've already seen.

In [ ]:
# flags2_threadpool.py

from collections import Counter
from concurrent.futures import ThreadPoolExecutor, as_completed

import httpx
import tqdm

from flags2_common import main, DownloadStatus
from flags2_sequential import download_one

# If the -m/--max_req command-line option is not given,
# this will be the maximum # of concurrent requests
DEFAULT_CONCUR_REQ = 30
# caps the maximum number of concurrent requests
# in order to prevent significant memory overhead
# caused by launching too many threads
MAX_CONCUR_REQ = 1000

def download_many(cc_list: list[str], base_url: str,
                  verbose: bool, concur_req: int) -> Counter[DownloadStatus]:
  counter: Counter[DownloadStatus] = Counter()
  with ThreadPoolExecutor(max_workers=concur_req) as executor:
    to_do_map = {} # this dict will map each Future instance with
                   # the respective country code for error reporting
    for cc in sorted(cc_list):
      future = executor.submit(download_one, cc, base_url, verbose)
      to_do_map[future] = cc # store the future and the country code in the dict
    done_iter = as_completed(to_do_map) # returns an iterator that yields futures as each task is done
    if not verbose:
      done_iter = tqdm.tqdm(done_iter, total=len(cc_list))
    # Iterate over the futures as they are completed
    for future in done_iter:
      try:
        # calling a result method on a future
        # either (1) returns the value returned by the callable
        #     or (2) raises whatever exception was caught when the callable was executed
        status = future.result()
      except httpx.HTTPStatusError as exc:
        error_msg = 'HTTP error {resp.status_code} - {resp.reason_phrase}'
        error_msg = error_msg.format(resp=exc.response)
      # Other network-related exceptions are handled here
      except httpx.RequestError as exc:
        error_msg = f'{exc} {type(exc)}'.strip()
      # Exit the loop if the user hits Ctrl-C
      except KeyboardInterrupt:
        break
      # If no exception escaped download_one, clear the error msg
      else:
        error_msg = ''

      if error_msg:
        status = DownloadStatus.ERROR
      counter[status] += 1
      if verbose and error_msg:
        cc = to_do_map[future] # to provide context for the error message
        print(f'{cc} error: {error_msg}')
  return counter

if __name__ == '__main__':
  main(download_many, DEFAULT_CONCUR_REQ, MAX_CONCUR_REQ)


Python threads are well suited for I/O intensive applications and the `concurrent.futures` package makes it relatively simple to use for certain use cases.

With `ProcessPoolExecutor`, you can also solve CPU-intensive problems on multiple cores.